# Day 11

## Part 1

- The monkeys operate based on how worried I am about each item
- Each monkey has the following attributes:
    - Starting items: a list of the worry level per item
    - Operation: how my worry level changes as the monkey inspects the item
    - Test: How the monkey decides whether/where to throw the item next
- After the operation but before the test the worry level is divided by 3
- Monkeys go in order of their number. When all the monkeys have gone, that is a round.
- Items are added to the end of a monkey slist of items.

`Figure out which monkeys to chase by counting how many items they inspect over 20 rounds. What is the level of monkey business after 20 rounds of stuff-slinging simian shenanigans?`

In [42]:
from utils import parse_from_file, ParseConfig

parser = ParseConfig('\n\n', str)

initial_state = parse_from_file('day_11.txt', parser)

print(initial_state[0])

Monkey 0:
  Starting items: 76, 88, 96, 97, 58, 61, 67
  Operation: new = old * 19
  Test: divisible by 3
    If true: throw to monkey 2
    If false: throw to monkey 3


In [43]:
# looks like we're in for some parser shenannigans!
parser = ParseConfig('\n\n', ParseConfig('\n', str))

initial_state = parse_from_file('day_11.txt', parser)

print(initial_state[0])

['Monkey 0:', '  Starting items: 76, 88, 96, 97, 58, 61, 67', '  Operation: new = old * 19', '  Test: divisible by 3', '    If true: throw to monkey 2', '    If false: throw to monkey 3']


In [44]:
parser = ParseConfig('\n\n', ParseConfig('\n', [
    ParseConfig(':', [ParseConfig(' ', [None, int]), None]),
    str, str, str, str, str
]))

initial_state = parse_from_file('day_11.txt', parser)

print(initial_state[0])

[[[0]], '  Starting items: 76, 88, 96, 97, 58, 61, 67', '  Operation: new = old * 19', '  Test: divisible by 3', '    If true: throw to monkey 2', '    If false: throw to monkey 3']


In [45]:
parser = ParseConfig('\n\n', ParseConfig('\n', [
    ParseConfig(':', [ParseConfig(' ', [None, int]), None]),  # Monkey index
    ParseConfig('Starting items:', [None, ParseConfig(', ', int)]),
    str, str, str, str
]))

initial_state = parse_from_file('day_11.txt', parser)

print(initial_state[0])

[[[0]], [[76, 88, 96, 97, 58, 61, 67]], '  Operation: new = old * 19', '  Test: divisible by 3', '    If true: throw to monkey 2', '    If false: throw to monkey 3']


In [46]:
parser = ParseConfig('\n\n', ParseConfig('\n', [
    ParseConfig(':', [ParseConfig(' ', [None, int]), None]),  # Monkey index
    ParseConfig('Starting items:', [None, ParseConfig(', ', int)]),
    ParseConfig('Operation: new = old ', [None, ParseConfig(' ', str)]),
    str, str, str
]))

initial_state = parse_from_file('day_11.txt', parser)

print(initial_state[0])

[[[0]], [[76, 88, 96, 97, 58, 61, 67]], [['*', '19']], '  Test: divisible by 3', '    If true: throw to monkey 2', '    If false: throw to monkey 3']


In [47]:
parser = ParseConfig('\n\n', ParseConfig('\n', [
    ParseConfig(':', [ParseConfig(' ', [None, int]), None]),  # Monkey index
    ParseConfig('Starting items:', [None, ParseConfig(', ', int)]),
    ParseConfig('Operation: new = old ', [None, ParseConfig(' ', str)]),
    ParseConfig('Test: divisible by', [None, int]),
    str, str
]))

initial_state = parse_from_file('day_11.txt', parser)

print(initial_state[0])

[[[0]], [[76, 88, 96, 97, 58, 61, 67]], [['*', '19']], [3], '    If true: throw to monkey 2', '    If false: throw to monkey 3']


In [48]:
parser = ParseConfig('\n\n', ParseConfig('\n', [
    ParseConfig(':', [ParseConfig(' ', [None, int]), None]),  # Monkey index
    ParseConfig('Starting items:', [None, ParseConfig(', ', int)]),
    ParseConfig('Operation: new = old ', [None, ParseConfig(' ', str)]),
    ParseConfig('Test: divisible by', [None, int]),
    ParseConfig('If true: throw to monkey', [None, int]),
    ParseConfig('If false: throw to monkey', [None, int]),
]))

initial_state = parse_from_file('day_11.txt', parser)

print(initial_state[0])

[[[0]], [[76, 88, 96, 97, 58, 61, 67]], [['*', '19']], [3], [2], [3]]


In [53]:
class Monkey:
    """A container for all the business attributed to one monkey"""
    def __init__(self, parser_dump: 'list[list]'):
        id, items, operation, test, on_true, on_false = parser_dump
        self.id = id[0][0]
        self.items = items[0]
        self.operation = self._parse_operation(*operation[0])
        self.test = self._parse_test(test[0])
        self.on_true = on_true[0]
        self.on_false = on_false[0]
    
    def _parse_operation(
            self, operator: str, operand: str) -> callable:
        """
        returns the operation as a function by parsing the values specified in
        the input.
        """
        def operation(value: int) -> int:
            if operand == 'old':
                op2 = value
            else:
                op2 = int(operand)
            if operator == '+':
                return value + op2
            elif operator == '*':
                return value * op2
            else:
                raise ValueError(f'operator not recognised: {operator}')
        
        return operation

    def _parse_test(self, divisor: int) -> callable:
        """
        returns a function that returns true if the an integer input is
        divisible by the divisor specified
        """
        def divisor_check(value: int) -> bool:
            return value % divisor == 0
        
        return divisor_check
    
    def __str__(self) -> str:
        string = '<Monkey: '
        attributes = []
        for attribute in dir(self):
            if not attribute.startswith('_'):
                attributes.append(f'{attribute}={getattr(self, attribute)}')
        string = string + ', '.join(attributes) + '>'
        return string

monkeys = [Monkey(state) for state in initial_state]

print(monkeys[0])

<Monkey: id=0, items=[76, 88, 96, 97, 58, 61, 67], on_false=3, on_true=2, operation=<function Monkey._parse_operation.<locals>.operation at 0x0000016EC7E731A0>, test=<function Monkey._parse_test.<locals>.divisor_check at 0x0000016EC7E71E40>>
